# Create dataset

In [1]:
import pandas as pd
import os

In [2]:
dataPath = os.getcwd() + '/court case data/testdata/'
caseCount = len(os.listdir(dataPath))
data = []
try:
    os.remove(dataPath + ".DS_Store")
except:
    print("No file DS_Store")
for filename in os.listdir(dataPath):
    f = open(os.path.join(dataPath, filename), encoding='utf-8')
    data.append([filename.replace('.txt', ''), f.read()])

verdict_df = pd.DataFrame(data, columns=["id", "case text"])
cases_df = pd.read_csv('./court case data/testdata.csv')
merged_df = cases_df.join(verdict_df.set_index('id'), on='id', how='left')

merged_df["verdict_date"] = pd.to_datetime(merged_df["verdict_date"])
merged_df["publication_date"] = pd.to_datetime(merged_df["publication_date"])

No file DS_Store


In [3]:
merged_df

,id,verdict_date,publication_date,verdict_type,jurisdiction_type,case text
0,ECLI-NL-RBNNE-2021-5018,2021-01-10,2021-11-23,uitspraak,['Strafrecht'],\n\nRECHTBANK NOORD-NEDERLAND\nAfdeling strafr...
1,ECLI-NL-RBZUT-2003-AH9598,2003-03-06,2003-09-07,uitspraak,['Strafrecht'],\n\nRECHTBANK ZUTPHEN\nMeervoudige economische...
2,ECLI-NL-RBZWB-2020-2646,2020-06-23,2020-06-23,uitspraak,['Strafrecht'],\n\nRECHTBANK ZEELAND-WEST-BRABANT\n\nStrafrec...
3,ECLI-NL-GHAMS-2019-1601,2019-08-05,2019-07-26,uitspraak,['Strafrecht'],\n\nafdeling strafrecht\nparketnummer: 23-0017...
4,ECLI-NL-GHAMS-2019-1602,2019-08-05,2019-07-26,uitspraak,['Strafrecht'],\n\nafdeling strafrecht\nparketnummer: 23-0017...
...,...,...,...,...,...,...
18119,ECLI-NL-RBAMS-2013-1294,2013-01-29,2013-08-10,uitspraak,['Strafrecht'],\n\nRECHTBANK AMSTERDAM\n\n\nVONNIS\n\n \n\n13...
18120,ECLI-NL-PHR-2020-1106,2020-11-24,2020-11-24,conclusie,['Strafrecht'],\n\nPROCUREUR-GENERAAL\n\n\nBIJ DE\n\n\nHOGE R...
18121,ECLI-NL-GHAMS-2017-2618,2017-06-29,2017-05-07,uitspraak,['Strafrecht'],\n\n\nparketnummer: 23-001217-13\ndatum uitspr...
18122,ECLI-NL-RBAMS-2013-BZ0392,2013-01-29,2013-01-02,uitspraak,['Strafrecht'],\nRECHTBANK AMSTERDAM \nVONNIS \n\n13/529144-...


# Split documents

### Old

In [ ]:
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|mr|mevr|mvr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|nl)"
articles = "[artikel ][0-9][.][0-9]"

def split_into_sentences2(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(articles,"[artikelnummer]",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

### New

In [4]:
import re

In [5]:
def split_into_sentences(text):
    sentences = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])|\n', text)
    sentences = [x for x in sentences if len(x) > 1]
    return sentences


Two ways of splitting the documents:
    For Word2Vec, we need sentences to be an array of words.
    For the rest, just the sentence is enough.

In [6]:
sentence_list_by_word = []
sentence_list = []

for i in range(len(merged_df)):
    doc = merged_df.iloc[i]['case text']
    sentences = split_into_sentences(doc)
    sentence_list.append(sentences)
    for j in sentences:
        word_list = [x for x in j.lower().rstrip().replace('.', '').split(' ') if len(x)>0]
        sentence_list_by_word.append(word_list)
        
print(len(sentence_list))
print(len(sentence_list_by_word))
# print(sentence_list[8])

18124
5107544


Testers:

In [7]:
for i in sentence_list[:1]:
    for j in i:
        print(j, '\n')

RECHTBANK NOORD-NEDERLAND 

Afdeling strafrecht  

Locatie Leeuwarden 

parketnummer 18/048259-21 

Vonnis van de meervoudige kamer voor de behandeling van strafzaken d.d. 1 oktober 2021 in de zaak van het openbaar ministerie tegen de verdachte  

      [verdachte]  

geboren op [geboortedatum] 1971 te [geboorteplaats]  

thans gedetineerd te [instelling]. 

Dit vonnis is gewezen naar aanleiding van het onderzoek ter terechtzitting van 17 september 2021. 

Verdachte is verschenen bijgestaan door mr. 

A.C. 

Huisman advocaat te Deventer. 

Het openbaar ministerie is ter terechtzitting vertegenwoordigd door mr. 

E.R. 

Jepkema. 

Tenlastelegging 

Aan verdachte is na nadere omschrijving van de tenlastelegging ten laste gelegd dat:  

1. 

hij in of omstreeks de periode van 1 oktober 2020 tot en met 19 februari 2021 meerdere malen althans eenmaal telkens te Minnertsga gemeente Waadhoeke tezamen en in vereniging met een of meer anderen althans alleen opzettelijk heeft bereid en/of bewerk

In [71]:
for i in sentence_list_by_word[:1]:
    for j in i:
        print(j, '\n')

rechtbank 

noord-nederland 



In [72]:
print(len(sentence_list))

18124


In [73]:
print(len(sentence_list_by_word))

5107544


# Create Word2Vec model

In [8]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

Create and save model

In [9]:
dutch_word2vec_model = Word2Vec(sentences=sentence_list_by_word, vector_size=100, window=5, min_count=1, workers=4)
dutch_word2vec_model.save("word2vec_dutch_court_cases.model")

Load model

In [10]:
dutch_word2vec_model = Word2Vec.load("word2vec_dutch_court_cases.model")

In [11]:
sims = dutch_word2vec_model.wv.most_similar('smokkel', topn=100)
print([i[0] for i in sims])
# print(sims)

['export', 'import', 'invoer', 'doorvoer', 'verkoop', 'handel', 'aanvoer', 'versnijding', 'gesmokkelde', 'drugssmokkel', 'productie/verwerking', 'exporteren', 'transporteren', 'bronlanden', 'invoeren', 'verhandeling', 'leverantie', 'importeren', 'uithalen', 'binnenbrengen', 'cocaïnehandel', 'binnensmokkelen', 'doorlevering', 'smokkelen', 'overdracht', 'drugstransporten', 'fabricage', 'wederverkoop', 'bronland', 'levering', 'afleveringen', 'dealen', '(synthetische)', 'transporten', 'doorverkoop', 'aankoop', 'terugwinnen', 'verspreiding', 'straatverkoop', 'productie', 'produktie', 'georganiseerde', 'transsport', 'leveranciers', 'aankopen', 'transport', 'verhandelen', 'vervaardiging', 'vondst', 'verwerking', 'terugwinproces', 'versnijden', 'drugshandel', 'vermenging', 'overdrachten', 'organisatoren', 'grootschalige', 'kweek', 'doorlating', 'cocaïnesmokkel', 'distributie', 'soft', 'uithaal', 'straathandel', 'drugsroute', 'verkopers', 'hennepteelt', 'overdacht', 'hasjtransporten', 'suppress

### Create list of drugs, smuggle, quantity keywords with Word2Vec model

In [12]:
def create_word2vec_relevant_words(words, matches):
    word2vec_list = []
    for word in words:
        results = dutch_word2vec_model.wv.most_similar(word, topn=100)
        for i in results:
            word2vec_list.append(i[0])
            
    word2vec_list = list(set([i for i in word2vec_list if word2vec_list.count(i)>matches]))
    return word2vec_list

Drugs list:

In [13]:
list_of_drugs = ['xtc', 'mdma', 'cocaine', 'wiet', 'speed', 'bmk', 'pmk']
word2vec_drug_list = create_word2vec_relevant_words(list_of_drugs, 2)

print(len(word2vec_drug_list))
print(word2vec_drug_list)

79
['mdma-hcl', 'speed', 'hasjiesj', 'aceton', 'ecstasy', 'crack', 'hennep', 'xtc', 'mdma-tabletten', 'morfine', 'xtc-tabletten', 'lidocaïne', 'crystal', 'tolueen', 'weed', 'amfetamine)', 'diazepam', 'amfetamine', 'heroïne;', 'pasta', 'xtc/mdma', 'tabletten', 'mdma', 'harddrugs', 'mefedron', 'cafeïne', 'mdma-pillen', 'lsd', 'amfetaminepasta', 'temazepam', 'meth', 'drugs', '34-methyleendioxymethamfetamine', 'manitol', 'speed/amfetamine', 'opium', 'zwavelzuur', 'metamfetamine', 'kristallen', 'olie', 'methadon', 'pillen)', 'cannabis', 'gbl', 'oxazepam', 'amfetamine/speed', 'heroïne)', 'hasj', 'amfetamineolie', 'methanol', '(mdma)', 'paracetamol', 'pillen', 'xtc-pillen', 'pep', 'cocaïne', 'coke', '2c-b', 'hashish', 'n-formylamfetamine', 'fenacetine', 'amfetaminesulfaat', 'heroïne', 'hash', 'ketamine', 'cocaïne;', 'ghb', 'methamfetamine', 'poedervorm', 'amfetaminen', '(mdma', 'mdma-kristallen', 'cocaïne?', 'azijnzuuranhydride', 'marihuana', 'mapa', 'eindproduct', 'mdma-poeder', 'amfetamineb

In [15]:
drugs_to_exclude = ['pasta', 'kristallen', 'poedervorm']
word2vec_drug_list = [drug for drug in word2vec_drug_list if drug not in drugs_to_exclude]
print(len(word2vec_drug_list))

76


Smuggle keyword list:

In [22]:
list_of_smuggle_words = ['smokkel', 'invoer', 'uitvoer', 'import', 'export', 'transport']
word2vec_smuggle_list = create_word2vec_relevant_words(list_of_smuggle_words, 3)

word2vec_smuggle_list = list(set(word2vec_smuggle_list + list_of_smuggle_words ))
print(len(word2vec_smuggle_list))
print(word2vec_smuggle_list)

45
['vervaardiging', 'gesmokkelde', 'levering', 'transporten', 'uithalen', 'produktie', 'organisatoren', 'hennepteelt', 'leverantie', 'invoeren', 'aankoop', 'handel', 'invoer', 'productie', 'importeren', 'transsport', 'bewerking', 'kweek', 'bronland', 'terugwinnen', 'in-/uitvoer', 'leverancier', 'verhandeling', 'smokkel', 'leveranciers', 'cocaïnehandel', 'transporteren', 'doorverkoop', 'exporteren', 'straatverkoop', 'export', 'versnijding', '(grootschalige)', 'uitvoer', 'doorvoer', 'versnijden', 'productie/verwerking', 'fabricage', 'transport', 'bronlanden', '(invoer', 'doorlevering', 'verkoop', 'aanvoer', 'import']


In [ ]:
# drugs_to_exclude = ['pasta', 'kristallen', 'poedervorm']
# word2vec_drug_list = [drug for drug in word2vec_drug_list if drug not in drugs_to_exclude]
# print(len(word2vec_drug_list))

Quantity keyword list:

In [21]:
list_of_quantity_words = ['tabletten', 'kilo', 'gram', 'pakketten']
word2vec_quantity_list = create_word2vec_relevant_words(list_of_quantity_words, 2)

word2vec_quantity_list = list(set(word2vec_quantity_list + list_of_quantity_words))
print(len(word2vec_quantity_list))
print(word2vec_quantity_list)

32
['kg)', 'tabletten', 'mg)', 'mdma-pillen', 'milliliter', 'blokken', 'gram)', 'gram;', 'ton', 'ponypacks', 'bollen', 'balen', 'mg', 'gripzakjes', 'stuks', 'cm)', 'kilo', 'kilogram', 'bolletjes', 'joints', 'pillen', 'pakjes', 'kilogram)', 'drums', 'wikkels', 'xtc-pillen', 'zakjes', 'gram', 'liter', 'pakketten', 'kilo)', 'planten)']


Country list:

In [23]:
list_of_countries = ['duitsland', 'colombia', 'alicante']
word2vec_country_list = create_word2vec_relevant_words(list_of_countries, 1)

print(len(word2vec_country_list))
print(word2vec_country_list)

97
['congo', 'parijs', 'pakistan', 'luxemburg', 'venezuela', 'dubai', 'zweden', 'dominicaanse', 'tsjechië', 'oostenrijk', 'brussel', 'slowakije', 'argentinië', 'peru', 'guayaquil', 'marokko', 'bosnië', 'kroatië', 'denemarken', 'kinshasa', 'sao', 'hongarije', 'litouwen', 'belgië', 'china', 'suriname', 'caracas', 'santa', 'australië', 'syrië', 'buitenland', 'curaçao', 'bulgarije', 'nederland', 'sierra', 'singapore', 'duitsland)', 'sevilla', 'berlijn', 'frankrijk', 'trinidad', 'canada', 'hamburg', 'rabat', 'roemenië', 'ecuador', 'hannover', 'paulo', 'jamaica', 'thailand', 'antwerpen', 'engeland', 'tanger', 'lissabon', 'londen', 'chili', 'nigeria', 'madrid', 'europa', 'spanje', 'dover', 'polen', 'leone', 'brazilië', 'bogota', 'afrika', 'natal', 'portugal', 'mexico', 'noorwegen', 'paramaribo', 'republiek', 'rica', 'italië', 'groot-brittannië', 'costa', '[land]', 'zwitserland', 'irak', 'bremen', 'oekraïne', 'frankfurt', 'turkije', 'lima', 'zuid-amerika', 'rusland', 'panama', 'griekenland', '

# Create SpaCy model

In [24]:
import spacy
from spacy import displacy
# !python -m spacy download nl_core_news_md
nlp = spacy.load('nl_core_news_md')


In [27]:
my_file = open("drugs list.txt", "r", encoding='utf-8')
my_file = my_file.readlines()
drugs_list = []
for i in my_file:
    drugs_list.append(i.replace('\n', ''))
drugs_list = list(set(drugs_list + word2vec_drug_list))
    
my_file = open("countries list.txt", "r", encoding='utf-8')
my_file = my_file.readlines()
countries_list = []
for i in my_file:
    countries_list.append(i.replace('\n', ''))
countries_list = list(set(countries_list + word2vec_country_list))

In [28]:
def configure_spacy_model():
    # Create dict of drug pattern and quantity pattern
    pattern_list = []
    
    for i in drugs_list:
        pattern_list.append({"label": "DRUG", "pattern": [{"lower": i}]})
    
#     quantity_rule = {"label": "QUANTITY", "pattern": [{"IS_DIGIT": True}, {"LOWER": "gram"}]}
#     pattern_list.append(quantity_rule)
    for i in word2vec_quantity_list:
        pattern_list.append({"label": "QUANTITY", "pattern": [{"IS_DIGIT": True}, {"LOWER": i}]})
        pattern_list.append({"label": "QUANTITY", "pattern": [{"ENT_TYPE": "CARDINAL"}, {"LOWER": i}]})
    
    for i in countries_list:
        pattern_list.append({"label": "GPE", "pattern": [{"lower": i.replace(' ', '').lower()}]})
    
    # Add drug and quantity rules to the model
    config = {
   "phrase_matcher_attr": None,
   "validate": True,
   "overwrite_ents": True,
   "ent_id_sep": "||",
    }
    ruler = nlp.add_pipe("entity_ruler", config=config)

    #List of Entities and Patterns
#     patterns = drugs_ent_list
    ruler.add_patterns(pattern_list)

In [29]:
configure_spacy_model()

# Select cases and chunks to keep

In [30]:
print(len(sentence_list) == len(merged_df))

True


For every case, split the sentences. If a sentence in a case contains a drug, a smuggle word, and a location: keep chunk and save to trafficking_df

In [31]:
# relevant_chunk_list = []
# ecli_list = []

# for index, case in enumerate(sentence_list):
#     chunk_list = []
#     trafficking_related = False
#     for chunk in case:
#         if any(drug in chunk for drug in word2vec_drug_list) and any(smuggle_word in chunk for smuggle_word in word2vec_smuggle_list):
#             ents = nlp(chunk).ents
#             if any(ent.label_ == "GPE" or ent.label_ == "LOC" for ent in ents):
#                 trafficking_related = True
#                 chunk_list.append(chunk)
#     if trafficking_related:
#         relevant_chunk_list.append(chunk_list)
#         ecli_list.append(merged_df.iloc[index]['id'].replace('-', ':'))

# trafficking_df = pd.DataFrame({'id': pd.Series(ecli_list), 'chunks': pd.Series(relevant_chunk_list)})                

In [78]:
relevant_chunk_list = []
ecli_list = []

for index, case in enumerate(sentence_list):
    chunk_list = []
    trafficking_related = False
    for chunk in case:
        word_list = [x for x in chunk.lower().rstrip().replace('.', '').split(' ') if len(x)>0]
        if any(drug in word_list for drug in word2vec_drug_list) and any(smuggle_word in word_list for smuggle_word in word2vec_smuggle_list):
            ents = nlp(chunk).ents
            if any(ent.label_ == "GPE" or ent.label_ == "LOC" for ent in ents):
                trafficking_related = True
                chunk_list.append(chunk)
    if trafficking_related:
        relevant_chunk_list.append(chunk_list)
        ecli_list.append(merged_df.iloc[index]['id'].replace('-', ':'))

trafficking_df = pd.DataFrame({'id': pd.Series(ecli_list), 'chunks': pd.Series(relevant_chunk_list)})       

In [79]:
print(f"{len(trafficking_df)} cases kept from original {len(merged_df)} cases.")


2590 cases kept from original 18124 cases.


# Create rule-based NER & POS tagging model

### Old

In [249]:
def extract_chunk_info(txt):
    source_country = None
    total_info = []
    for token in nlp(txt):
        info = {}
        drug_info = {}
        countries = []
        
        if token.ent_type_ == "DRUG":
            info = {"drug": token.text}
            
            ## Get source and destination
            for ancestor in token.ancestors:
                for nephew in ancestor.children:
                    if nephew.ent_type_ == "GPE" or nephew.ent_type_ == "LOC":
                        countries.append(nephew)
                        for child in nephew.children:
                            if child.dep_ == "conj" and child.ent_type_ == "GPE" or child.ent_type_ == "LOC":
                                countries.append(child.text)
                            elif child.pos_ == "ADP" and child.dep_ == "case":
                                adj = child.text
            if len(countries) > 0 :
                try:
                    info[adj] = countries
                except:
                    info['land'] = countries
                        
            ## Get volume
            for ancestors in token.ancestors:
                for nephew in ancestors.children:
                    if nephew.ent_type_ == "QUANTITY" or nephew.ent_type_ == "CARDINAL":
                        for second_nephew in nephew.children:
                            if second_nephew.is_digit != nephew.is_digit:
                                if second_nephew.is_digit:
                                    info['volume'] = second_nephew.text
                                    info['volume_type'] = nephew.text
                                else:
                                    info['volume'] = nephew.text
                                    info['volume_type'] = second_nephew.text
            if 'volume' not in info:
                for child in token.children:
                    if (child.dep_ == "det" and child.like_num) or (child.dep_ == "nummod"):
                        info['volume'] = child.text
                                
        if len(info) > 1:
#             print(info)
            total_info.append(info)
    return total_info

### New

In [80]:
# Get linguistic distance between token a and token b. After iter 10 it is deemed a too far distance.
def get_linguistic_distance(a, b):
    tokens_to_consider = [b]
    found = False
    iters = 0
    while not found:
        for token in tokens_to_consider:
            tokens_to_add = []
            for ancestor in token.ancestors:
                if ancestor not in tokens_to_add and ancestor not in tokens_to_consider:
                    tokens_to_add.append(ancestor)
            for child in token.children:
                if child not in tokens_to_add and child not in tokens_to_consider:
                    tokens_to_add.append(child)
            tokens_to_consider = tokens_to_consider + tokens_to_add
        for x in tokens_to_consider:
            if a.orth == x.orth:
                found = True
        iters += 1
        if iters == 10:
            found = True
    return iters

In [81]:
def get_adposition_from_loc(token):
    for child in token.children:
        if child.pos_ == "ADP" and child.dep_ == "case":
            return child.text

In [174]:
def extract_chunk_info(txt):
    result = {}
    print('START CHUNK')
    for token in nlp(txt):
        if token.ent_type_ == "DRUG":
            print(f" \n Extracting info for {token.text}.")
            drug_info = extract_info_from_drug(token, txt)
            result[token.text] = drug_info
    return result
            
            
def extract_info_from_drug(drug, txt):
    volumes = []
    locations = {}
    irrelevant_locations = []
    for token in nlp(txt):
        
        # Extract countries
#         if token.ent_type_ == "GPE" or token.ent_type_ == "LOC":
        if token.ent_type_ == "GPE":
            dist = get_linguistic_distance(drug, token)
#             if dist < 15:
            adj = get_adposition_from_loc(token)
            print(f"    {adj}: {token.text}, dist: {dist}, conj: {token.conjuncts}")
            locs = [token.text]
            for loc in token.conjuncts:
                locs.append(loc.text)
            if adj not in locations:
                locations[adj] = locs
            else:
                for loc in locs:
                    if loc not in locations[adj]:
                        locations[adj].append(loc)
#             else:
#                 irrelevant_locations.append(token.text)
#                 print(f"{token.text} is irrelevant.")
        
        # Extract volume
        if token.ent_type_ == "QUANTITY":
            volume = {}
            dist = get_linguistic_distance(drug, token)
            second_token = ""
#             if dist < 10:
            quantity = {}
            for ancestor in token.ancestors:
                if ancestor.ent_type_ == "QUANTITY":
                    second_token = ancestor
            for child in token.children:
                if child.ent_type_ == "QUANTITY":
                    second_token = child

            ## Decide volume and volume_type
            if nlp(token.text)[0].ent_type_ == "CARDINAL":
                volume['volume'] = token.text
                volume['volume_type'] = second_token.text
            elif nlp(second_token.text)[0].ent_type_ == "CARDINAL":
                volume['volume'] = second_token.text
                volume['volume_type'] = token.text
            volume['dist'] = dist

            #Only append when not already in volumes
            if volume not in volumes:
                volumes.append(volume)
#             else:
#                 print(f"{token.text} is irrelevant.")
                
        
        
            
    print(volumes)

    result = {}
    if bool(locations):
        result['locations'] = locations
#     if len(volumes) > 0:
    result["volume"] = volumes
    if len(irrelevant_locations) > 0:
        result["irrelevant_locations"] = irrelevant_locations
    
    return result
    
        


# Results

Task now: make sure the result from the extract_chunk_info is correct

In [175]:
for i in range(len(trafficking_df[:10])):
    chunks = trafficking_df.iloc[i]['chunks']
    id = trafficking_df.iloc[i]['id']
    print(id)
    for chunk in chunks:
        info = extract_chunk_info(chunk)
        print(f"Info: {info} \n")

ECLI:NL:RBZUT:2003:AH9598
START CHUNK
 
 Extracting info for aceton.
    None: Haarlem, dist: 3, conj: (Amstelveen,)
    None: Amstelveen, dist: 3, conj: (Haarlem,)
[]
 
 Extracting info for drugs.
    None: Haarlem, dist: 4, conj: (Amstelveen,)
    None: Amstelveen, dist: 4, conj: (Haarlem,)
[]
Info: {'aceton': {'locations': {None: ['Haarlem', 'Amstelveen']}, 'volume': []}, 'drugs': {'locations': {None: ['Haarlem', 'Amstelveen']}, 'volume': []}} 

ECLI:NL:RBZWB:2020:2646
START CHUNK
 
 Extracting info for BMK.
    None: IBC-, dist: 10, conj: ()
[{'volume': '241', 'volume_type': 'liter', 'dist': 2}, {'dist': 1}, {'volume': '8', 'volume_type': 'liter', 'dist': 2}, {'volume': '15', 'volume_type': 'liter', 'dist': 2}, {'volume': '5', 'volume_type': 'liter', 'dist': 2}, {'volume': '14', 'volume_type': 'liter', 'dist': 2}, {'dist': 10}]
 
 Extracting info for Benzylmethylketon.
    None: IBC-, dist: 10, conj: ()
[{'volume': '241', 'volume_type': 'liter', 'dist': 10}, {'dist': 10}, {'volume'

    in: Rotterdam, dist: 6, conj: ()
[]
Info: {'cocaïne': {'locations': {'in': ['Rotterdam']}, 'volume': []}} 

START CHUNK
 
 Extracting info for cocaïne.
    in: Rotterdam, dist: 3, conj: ()
[]
Info: {'cocaïne': {'locations': {'in': ['Rotterdam']}, 'volume': []}} 

START CHUNK
 
 Extracting info for cocaïne.
    None: I.6, dist: 10, conj: ()
[]
Info: {'cocaïne': {'locations': {None: ['I.6']}, 'volume': []}} 

START CHUNK
 
 Extracting info for cocaïne.
    in: Nederland, dist: 1, conj: ()
[]
Info: {'cocaïne': {'locations': {'in': ['Nederland']}, 'volume': []}} 

ECLI:NL:RBAMS:2017:9086
START CHUNK
 
 Extracting info for cocaïne.
    None: I.6, dist: 10, conj: ()
[]
Info: {'cocaïne': {'locations': {None: ['I.6']}, 'volume': []}} 

START CHUNK
 
 Extracting info for cocaïne.
    in: Nederland, dist: 1, conj: ()
[]
Info: {'cocaïne': {'locations': {'in': ['Nederland']}, 'volume': []}} 

ECLI:NL:RBAMS:2017:9088
START CHUNK
 
 Extracting info for cocaïne.
    in: Rotterdam, dist: 6, conj: 

# Sandbox

In [120]:
trafficking_df

,id,chunks
0,ECLI:NL:RBZUT:2003:AH9598,[hij op tijdstippen in de periode 6 februari 2...
1,ECLI:NL:RBZWB:2020:2646,[- een (compleet) in werking zijnde laboratori...
2,ECLI:NL:PHR:2007:BA1113,[In de maand december 1998 ontstond onder ande...
3,ECLI:NL:GHAMS:2018:2662,[Dit feit heeft betrekking op een geslaagde in...
4,ECLI:NL:RBAMS:2017:9087,[Verdachte heeft in de ochtend van 5 april 201...
...,...,...
2585,ECLI:NL:GHSHE:2020:1730,[Gezien het vorenstaande is het hof van oordee...
2586,ECLI:NL:HR:1998:ZD1191,"[""4. hij in de periode van 1 januari 1993 tot ..."
2587,ECLI:NL:RBNHO:2013:10924,[Met de officier van justitie en de raadsvrouw...
2588,ECLI:NL:GHSHE:2021:3205,[hij verdachte in of omstreeks de periode van ...


In [147]:
case = list(trafficking_df[trafficking_df['id'] == 'ECLI:NL:GHAMS:2018:2662']['chunks'])[0]

In [148]:
for chunk in case:
    displacy.render(nlp(chunk), style = 'ent')
    extract_chunk_info(chunk)

START CHUNK
 
 Extracting info for cocaïne.
    vanuit: Argentinië, dist: 2, conj: ()
[{'volume': '900', 'volume_type': 'gram'}]


In [ ]:
def get_entities(txt):
    doc = nlp(txt)
    for ent in doc.ents:
        print(ent, ent.label_)

Get all loc entities

In [186]:
def list_entities(entity):
    entity_list = []
    for i in range(len(trafficking_df)):
        chunks = list(trafficking_df.iloc[i]['chunks'])
        for chunk in chunks:
            doc = nlp(chunk)
            for ent in doc.ents:
                if ent.label_ == entity:
#                     print(trafficking_df.iloc[i]['id'])
                    if ent.text not in entity_list:
                        entity_list.append(ent.text)
    return entity_list

In [187]:
loc_list = list_entities("LOC")


In [188]:
print(loc_list)

['gaswasser', 'Noordzee', 'Ettenseweg', 'Klein Horendonk', 'Maasdelta', 'Rotterdamse haven', 'medeveroordeelde', 'Prins Clausstraat', 'Westerschelde', 'Daartegenover', 'Wvmc', 'NFI-onderzoek', 'woning.2', 'inrichting/opbouw', 'Verderop', 'ZD04', 'Westelijke Havendijk', 'IJmond', 'Leuckartmethode', 'Eindhovenseweg', 'Zuid', 'koeriersauto', 'Tonsdijk', 'mengsels/substanties', 'Zuid-Limburgse', 'Industrieweg', 'Oost', 'micro-cellulose', 'Opiumwetbesluit', 'Wieringerwaardstraat', 'Luikerweg', 'verwijzingsbevel', 'Pijp', 'Lier', 'auditu-verklaringen', 'Keton', 'Rijn', 'Josemans-arrest', 'Bouchereau-arrest', 'schuur.5', 'Vogelweide', 'Ermelo4', 'Stationsweg', 'beek', 'Roteb', 'Gezinsherenigingsrichtlijn', 'Korvelseweg', 'Berkel', 'Arnhem', 'reactieketels L2', 'Gravesend', 'Sint Hubertuslaan', 'Tweede Groenedijk', 'Ermerweg', 'Diemen-Zuid', 'Noord-Beemster', 'Douanelaboratorium', 'Efedrine', 'enkelslag tabletteermachine', 'Einlassung', 'hennepkweek', 'AE-1-2017', 'Spaanse Polder', 'Nagel-onde

In [ ]:
gpe_list = list_entities("GPE")
print(gpe_list)

In [191]:
for i in gpe_list:
    print(i)

Haarlem
Amstelveen
IBC-
Gemeenschap
Nederland
Marokko
Argentinië
Rotterdam
I.6
Jamaica
Hoogeveen
gemeente Hoogeveen
Utrecht
Frankrijk
België
gemeente Tytsjerksteradiel
bevorderingshandelingen
Duitsland
Bonn
Landgraaf
Oud-Gastel
Breda
buitenland
Curaçao
Leende
Leuckartmethode
Engeland
Bergschenhoek
Brabant
Zuid-Oost
€
Rijsbergen
Wernhout
N-ethyl
Roden
Leek
Drachten
Tolbert
Veluwerand
Den Haag
Tilburg
Schiphol
Roermond
Sprundel
het/die
Essen
piperonal
giftstoffen
Neede
gemeente Berkelland
Borculo
Leuven
Kockengen
baksystemen
Nederhorst
Putten
Zeewolde
Neede/Borculo
Ammoniumchloride
Baarn
MDMA‑kristallen
tabletteren
Caldic
ontvreemden
Maassluis
Roosendaal
Langdonk
MDMA-base
Reek
gemeente Valkenswaard
verdachte9
gemeente Ermelo
Zundert
Stevensbeek
Neede/
namen
Mallorca
Uden
Beverwijk
Haler
RVS:2013:BZ8430
Turnhout
daartoe/daarover
Oisterwijk
Oost-Nederland
Dominicaanse
Republiek
Sky-ID
Amsterdam
Antwerpen
Brussel
Medeveroordeelde
Noord- en
Apeldoorn
Zuid-Amerika
Ecuador
Italië
Luik
Best
la